# Face recognizer using pre-trained VGG-Face model

This note book is inspired by Sefik Ilkin Serengil's [Deep Face Recognition with Keras](https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/) implementation. His code can be found at 
https://github.com/serengil/tensorflow-101/blob/master/python/vgg-face.ipynb

Sefik's original notebook was updated to use the tf.keras submodule instead of the standalone keras pacakage.
tf.keras was introduced in TensorFlow v1.10.0, this is the first step in integrating Keras directly within the
TensorFlow package itself. 

The [Deep Face Recognition](http://www.robots.ox.ac.uk/~vgg/publications/2015/Parkhi15/parkhi15.pdf) VGG-Face CNN architecture along with its pre-trained weights were published by the Visual Geometry Group of University of Oxford [here](https://www.robots.ox.ac.uk/~vgg/software/vgg_face/). This notebook will use the pre-trained VGG-Face model to recognize whether the two pictures of persons presented are the same or different person. 

## Setup to access data from IBM Object Storage

The following file will be downloaded from IBM Object Storage:
* vgg_face_weights.h5
* jolie1-c.jpg
* jolie3-c.jpg
* jolie3-c.jpg
* jolie4-c.jpg
 

In [ ]:
!ls -o

In [ ]:
# Run the command if ibm_boto3 is not installed.
!pip install ibm-cos-sdk

In [ ]:
# Install the boto library.
import ibm_boto3
from ibm_botocore.client import Config

cos_credentials = {
  "apikey": "bzaovcHjJb4vRir7CzWw2zopFAKVACb21WKlvSp2EeEv",
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key 155a3956-0e11-4df1-9228-f00ff0090ea6",
  "iam_apikey_name": "vggh5",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/d17d7328be3c182a41e9856d62e02770::serviceid:ServiceId-974be549-3a00-4101-80aa-8414ef94a269",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/d17d7328be3c182a41e9856d62e02770:21403250-3385-472c-bc43-97fc64697887::"
}

In [ ]:
# Define endpoint information.
# The service_endpoint can be copied from IBM Cloud as following:
# * From IBM Cloud Resource list ( https://cloud.ibm.com/resources ), click on the target storage service.
# * Click on the target bucket, then click on the Configuration tab on the left menu.
# * Copy the string displace under the public endpoint

service_endpoint = 'https://s3.us-east.cloud-object-storage.appdomain.cloud'

# Define the authorization endpoint.
auth_endpoint = 'https://iam.bluemix.net/oidc/token'

# Create a COS resource.
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [ ]:
import os

def download_file_from_cos(bucket_obj, source_filenames, dest_filenames):
    for i in range(len(source_filenames)):
        if not os.path.isfile(dest_filenames[i]):
            print("Downloading", source_filenames[i])
            bucket_obj.download_file(source_filenames[i],dest_filenames[i])
        else:
            print(source_filenames[i], "was downloaded previously." )

# The bucket object that has the h5 file.
bucket_obj = cos.Bucket('vggh5')
            
# Files to download from object storage
source_filenames = ["vgg_face_weights.h5", 'jolie1-c.jpg', 'jolie2-c.jpg', 'jolie3-c.jpg', 'jolie4-c.jpg']
dest_filenames = source_filenames

download_file_from_cos(bucket_obj, source_filenames, dest_filenames)

## Import the prerequisite libraries

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

## Contruct the neural network model

VGG-Face has 22 layers and 37 deep units. Here is how Sefik visualized the VGG-Face architure.

![](img/vgg-face.JPG)

In [ ]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

## Loading the model with pretrained weights

Although the pre-trained weights can be downloaded from the Visual Geometry Group's [Website](https://www.robots.ox.ac.uk/~vgg/software/vgg_face/), but it is matlab compatible. 
The transformed pre-trained weights for Keras can be downloaded from this link 
https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing

In [ ]:
model.load_weights('vgg_face_weights.h5')

### Processing image

Images are preprocessed to transform them into numeric representation vectors of sizes that are expected by the VGG-Face model.
Notice that VGG model expects 224x224x3 sized input images. The 3rd dimension refers to number of channels or RGB colors. Besides, preprocess_input function normalizes input in scale of [-1, +1].

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [ ]:
picture = "jolie1-c.jpg"
img = preprocess_image(picture)
print("Image shape:", img.shape)
print(img)

plt.imshow(image.load_img((picture)))

### Model for similarity determination

Create a new face descriptor  model by omitting the last layer of the VGG-Face model. The output vectors of this model will be used as the face descriptor 
representations for similarity determination. 

Vectors similarity is measured by their distance. There are two common ways to find the distance of two vectors: cosine distance and euclidean distance. Cosine distance is
equal to 1 minus cosine similarity. No matter which measurement we adapt, they all serve for finding similarities between vectors.

In [ ]:
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [ ]:
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

### Verify face similarity

This function calculate similarity between the face images by passing the numerical representations of images to the neural 
network. The outputs are the "face descriptor" representation vectors. These vectors are then be used to compared with
each other by calculating cosine distance and/or euclidean distance.

In [ ]:
epsilon = 0.40

def verifyFace(img1, img2):
    img1_representation = vgg_face_descriptor.predict(preprocess_image((img1)))[0,:]
    img2_representation = vgg_face_descriptor.predict(preprocess_image((img2)))[0,:]
    
    cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
    euclidean_distance = findEuclideanDistance(img1_representation, img2_representation)
    
    print("Cosine similarity: ",cosine_similarity)
    print("Euclidean distance: ",euclidean_distance)
    
    if(cosine_similarity < epsilon):
        print("Persons in "+ img1 + " and " + img2 + " are the same person")
    else:
        print("Persons in "+ img1 + " and " + img2 + " are not the same person")
    
    f = plt.figure()
    f.add_subplot(1,2, 1)
    plt.imshow(image.load_img((img1)))
    plt.xticks([]); plt.yticks([])
    f.add_subplot(1,2, 2)
    plt.imshow(image.load_img((img2)))
    plt.xticks([]); plt.yticks([])
    plt.show(block=True)
    print("-----------------------------------------")

### Compare images

Finally, let's comnpare some iamges.

In [ ]:
verifyFace("jolie1-c.jpg", "jolie2-c.jpg")
verifyFace("jolie1-c.jpg", "jolie4-c.jpg")

### Compare your images

Now you can upload your own images to the IBM Cloud Object Storage, download them to this notebook and then compare them.
Does this face recognition program differentiate people correctly?

In [ ]:
#comma seperated name string
your_image_names=['image1.jpg','image2.jpg']

#your_image_names=['jolie3-c','jolie3-c']
dest_image_names=your_image_names
bucket_obj = cos.Bucket('vggh5')

download_file_from_cos(bucket_obj, your_image_names, dest_image_names)     

In [ ]:
verifyFace("image1.jpg", "image2.jpg")

In [ ]:
!rm *.jpg